# WebScrape

## Imports

In [1]:
# Standard
import pandas as pd
import numpy as np
import re
from time import sleep

# Selenium
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Functions

In [2]:
def spec_func(url, reference_number):
    """
    Takes a url where there is a list of tables and merges them together into one table after transposing each
    
    Parameters:
        url (str): the url where the tables are on
        reference_number (str): the reference number of the watch
        
    Returns:
        a table with a single row where the row contains data of column 1 of each table 
        and the column names are of column 0 of each table. 
    """
    specs = pd.read_html(url)
    table = pd.DataFrame(columns = ['reference_number'])

    for spec in specs:
        specd = spec.T
        specd.columns = specd.iloc[0]
        specd = specd.iloc[1:]
        
        specd['reference_number'] = reference_number
        table = pd.merge(table, specd, on = 'reference_number', how = 'outer')
    
    return table

## Navigation

In [3]:
driver = webdriver.Firefox()

In [7]:
driver.get('https://www.breitling.com/us-en/watches/all/')


# Close pop-up
try:
    close_pop = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.close-btn.mfp-close.icon.icon-close.mr-15'))
    )
    close_pop.click()
except:
    pass


# Sort Watches
sort = driver.find_elements(By.ID, 'sorting')[0]
sort_lh = driver.find_elements(By.CSS_SELECTOR, '.custom-checkbox-wrap.m-0.p-5.position-relative.pr-20')[2]
sort.click()
sort_lh.click()


# Get total number of watches
watch_tot = driver.find_elements(By.CSS_SELECTOR,
                     '.align-items-center.btn-watch-filters.d-flex.px-0.py-20')[2]\
            .text\
            .split(' ')[0]
watch_tot = int(watch_tot)


# Get total number of items per page
watch_sel = '.d-flex.flex-column.justify-content-center.position-relative.pt-5.px-5.text'
watch_len = len(driver.find_elements(By.CSS_SELECTOR, watch_sel))


# Get total number of pages as int
tot_page = watch_tot // watch_len + 1


# Get list of watch urls
x = 0
links2 = []

while x <= tot_page:
    x += 1
    item_list_sel = driver.find_elements(By.CSS_SELECTOR, watch_sel)
    for item in item_list_sel:
        try:
            links2.append(item.find_element(By.TAG_NAME, 'a').get_attribute('href'))
        except:
            pass
    sleep(2)
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '.btn.m-10'))
        )
        next_button.click()
        sleep(1)
    except:
        pass
links = list(set(links2))

In [8]:
print(f'length original: {len(links2)}')
print(f'length de-dupe: {len(links)}')

length original: 318
length de-dupe: 302


## Scrape

In [9]:
def scrape_func(links = links):
    spec_df = pd.DataFrame(columns = ['reference_number'])
    x = 0
    for index, url in enumerate(links):
        driver.get(url)
        sleep(3)
        # reference_number
        try:
            reference_number = driver.find_element(By.CLASS_NAME,'head').text.split('\n')[0]
        except:
            reference_number = url

        # spec tables
        specs = pd.read_html(url)
        spec_row = spec_func(url, reference_number)

        # Parent Model
        try:
            pm =  driver.find_element(By.CSS_SELECTOR, '.breadcrumbs-mobile.breadcrumbs-not-shortened.mx-15-neg')\
            .find_elements(By.CLASS_NAME, 'text-primary-blue')[2]\
            .get_attribute('innerHTML')\
            .strip(' ')
            spec_row['parent_model'] = pm
        except:
            pass

        # Specific Model
        try:
            spec_row['specific_model'] = driver.find_element(By.CLASS_NAME,'head').text.split('\n')[1]
        except:
            pass
        
        # Price and Currency
        try:
            p_c = driver.find_element(By.CSS_SELECTOR, '.version-price.font-heading.text-uppercase.text-primary-blue').text
            spec_row['currency'] = p_c.split(' ')[0]
            spec_row['price'] = p_c.split(' ')[1]
        except:
            pass
        
        # Image
        try:
            spec_row['image_URL'] = driver.find_element(By.CSS_SELECTOR, '.ratio-element.watch-image')\
                                .get_attribute('src')
        except:
            pass
        
        # URL
        spec_row['watch_URL'] = url
        
        # Description
        try:
            spec_row['description'] = driver.find_element(By.ID, 'version-description').text
        except:
            pass
        
        # Add to table
        spec_df = pd.concat([spec_df, spec_row], ignore_index = True, sort=False)
        print(f' finished {index}')
    return spec_df

In [10]:
spec_df = scrape_func(links)

 finishsed 0
 finishsed 1
 finishsed 2
 finishsed 3
 finishsed 4
 finishsed 5
 finishsed 6
 finishsed 7
 finishsed 8
 finishsed 9
 finishsed 10
 finishsed 11
 finishsed 12
 finishsed 13
 finishsed 14
 finishsed 15
 finishsed 16
 finishsed 17
 finishsed 18
 finishsed 19
 finishsed 20
 finishsed 21
 finishsed 22
 finishsed 23
 finishsed 24
 finishsed 25
 finishsed 26
 finishsed 27
 finishsed 28
 finishsed 29
 finishsed 30
 finishsed 31
 finishsed 32
 finishsed 33
 finishsed 34
 finishsed 35
 finishsed 36
 finishsed 37
 finishsed 38
 finishsed 39
 finishsed 40
 finishsed 41
 finishsed 42
 finishsed 43
 finishsed 44
 finishsed 45
 finishsed 46
 finishsed 47
 finishsed 48
 finishsed 49
 finishsed 50
 finishsed 51
 finishsed 52
 finishsed 53
 finishsed 54
 finishsed 55
 finishsed 56
 finishsed 57
 finishsed 58
 finishsed 59
 finishsed 60
 finishsed 61
 finishsed 62
 finishsed 63
 finishsed 64
 finishsed 65
 finishsed 66
 finishsed 67
 finishsed 68
 finishsed 69
 finishsed 70
 finishsed 71
 f

### Store

In [11]:
spec_df.to_csv('breitling_raw.csv')
# spec_df = pd.read_csv('breitling_raw.csv', index_col = 0)

## Wrangling

In [12]:
col_names = ['reference_number','watch_URL','type','brand','year_introduced',
             'parent_model','specific_model','nickname','marketing_name','style','currency',
             'price','image_URL','made_in','case_shape',
             'case_material', 'case_finish', 'caseback', 'diameter', 'between_lugs',
             'lug_to_lug', 'case_thickness', 'bezel_material', 'bezel_color', 'crystal',
             'water_resistance', 'weight', 'dial_color', 'numerals',
             'bracelet_material', 'bracelet_color', 'clasp_type', 
             'movement', 'caliber', 'power_reserve', 'frequency', 'jewels', 
             'features', 'short_description', 'description']

df = pd.DataFrame(columns = col_names)

### Listing Basics

In [13]:
df['reference_number'] = spec_df['reference_number']

df['watch_URL'] = spec_df['watch_URL']

df['brand'] = 'Breitling'

df['parent_model'] = spec_df['parent_model']

df['specific_model'] = spec_df['specific_model']

df['nickname'] = spec_df['specific_model']

# Marketing Name

# Style

df['currency'] = spec_df['currency']

df['price'] = spec_df['price']\
                .str\
                .replace(',','')\
                .astype('float')

df['image_URL'] = spec_df['image_URL']

df['made_in'] = 'Switzerland'

### Case Shape

In [14]:
def clean_water(x):
    """Makes the water pressure column in ATM"""
    bars = re.search(r'(\d+)\sbars', x)
    meters = re.search(r'(\d+)\sm', x)
    if bars:
        return f'{bars.group(1)} ATM'
    elif meters:
        return f'{int(meters.group(1))//10} ATM'
    else:
        return None

In [138]:
df['case_material'] = spec_df['Case material']

# case_finish

df['caseback'] = spec_df['Caseback']

df['diameter'] = spec_df['Diameter']

df['between_lugs'] = spec_df['Lug width (in-between lug)']

df['lug_to_lug'] = spec_df['Height (upper lug tip to lower lug tip)']

df['case_thickness'] = spec_df['Thickness']

#bezel_material

# bezel_color

df['crystal'] = spec_df['Crystal']\
                .astype(str).apply(lambda x: x.split(',')[0]).iloc[19]

df['water_resistance'] = spec_df['Water resistance'].map(clean_water)

df['weight'] = spec_df['Product Weight (Approx.)']

In [52]:
# Fix typo on their end
df.loc[df['reference_number'] == 'A17375211I1S1', ['case_thickness']] = '12.56 mm'

### Dial

In [ ]:
#Dial Color
#Numeral

### Bracelet / Strap

In [17]:
df['bracelet_material'] = spec_df['Strap material']

df['bracelet_color'] = spec_df.apply(lambda x:
                                    f"{x['Strap color']} {x['Strap material']}".title(), axis = 1)

df['clasp_type'] = spec_df.apply(lambda x:
                                 f"{x['Buckle size']} {x['Buckle Material']} \
                                 buckle, {x['Buckle type']}".replace('NaN','').lower(), axis = 1)

### Movement

In [18]:
def freq_func(x):
    """formats the frequency and ads the Hz and vph"""
    num = re.search(r'(\d+) v', x.replace(',', ''))
    if num:
        vph = int(num.group(1))
        hz = int(vph/7200)
        return f'{hz} Hz ({vph:,} vph)'

def power_reserve(x):
    """formats power reserve"""
    num = re.search(r'(\d+) h', x)
    if num:
        return f'{num.group(1)} hours'

In [19]:
df['movement'] = spec_df['Movement']

df['caliber'] = spec_df['Caliber']

df['power_reserve'] = spec_df['Power reserve']\
                        .astype(str)\
                        .map(power_reserve)

df['frequency'] = spec_df['Vibration'].astype(str).apply(freq_func)

df['jewels'] = spec_df['Jewel']\
                .astype(str)\
                .map(lambda x: x.split(' ')[0]\
                .replace('nan',''))

### Other

In [133]:
def desc_func(long_desc):
    """splits the long description by the first sentence, used for the short_description field."""
    short_desc = long_desc.split(r'\.\s')[0]
    return short_desc

def feature_func(row):
    """combines a few columns of interest into one column as a list. Also removes the NaN from list"""
    
    if type(row['Chronograph']) == str:
        chrono = f"Chronograph = {row['Chronograph']}"
    else:
        chrono = None
        
    if type(row['Calendar']) == str:
        calendar = f"Calendar = {row['Calendar']}"
    else:
        calendar = None
        
    if type(row['Bezel']) == str:
        bezel = f"Bezel = {row['Bezel']}"
    else:
        bezel = None
        
    if type(row['Crown']) == str:
        crown = f"Crown = {row['Crown']}"
    else:
        crown = None
        
    if type(row['Other functions']) == str:
        other = f"Other functions = {row['Other functions']}"
    else:
        other = None
    
    features = [chrono, calendar, bezel, crown, other]
    
    #remove None
    filtered_features = [x for x in features if x is not None]
    
    return filtered_features

In [134]:
#features
df['features'] = spec_df.apply(feature_func, axis = 1)

#short desc
df['short_description'] = spec_df['description'].apply(desc_func)

#descr
df['description'] = spec_df['description']

## Convert to CSV

In [139]:
df.to_csv('breitling_scrape2.csv', index = False)